<a href="https://colab.research.google.com/github/MarohBJoshua22/21st-Century-Movie-Recommender-for-kids/blob/main/BERT%20%26%20SVM%20Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install transformers torch scikit-learn pandas

In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertModel
import torch
from torch.utils.data import DataLoader, Dataset

In [34]:
 # Load the dataset from a CSV file
df = pd.read_csv('/content/drive/MyDrive/profanity_en.csv')

In [35]:
df = pd.read_csv('/content/drive/MyDrive/profanity_en.csv')
df_filtered = df[df['category_1'] == 'sexual anatomy / sexual acts']
df_filtered = df_filtered[['text']]
df_filtered['label'] = 1  # Binary label for grooming

In [36]:
df2 = pd.read_csv('/content/drive/MyDrive/Non-grooming_set_utf8.csv')
df2 = df2[['text', 'label']]  # Assume 'label' column exists

In [37]:
# Combine the two datasets
combined_df = pd.concat([df_filtered, df2], ignore_index=True)

# Convert 'label' column to numeric, handling non-numeric values
combined_df['label'] = pd.to_numeric(combined_df['label'], errors='coerce').fillna(0).astype(int)
# 'coerce' replaces non-numeric values with NaN, which are then filled with 0 and converted to integers.

In [38]:
combined_df.shape

(1188, 2)

In [39]:
combined_df.head(1000)

,text,label
0,69,1
1,@55,1
2,@ssfcker,1
3,@ssfucker,1
4,@ssfvcker,1
...,...,...
995,MRW,0
996,IDC,0
997,ILY,0
998,ADIH,0


In [40]:
# Split the data into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    combined_df['text'], combined_df['label'], test_size=0.2, random_state=42
)

In [41]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [42]:
# Function to encode text using BERT
def encode_texts(texts):
    encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**encodings)
    # Use the [CLS] token embedding for classification
    return outputs.last_hidden_state[:, 0, :].numpy()

In [ ]:
# Encode the training and test texts
train_embeddings = encode_texts(train_texts)
test_embeddings = encode_texts(test_texts)

In [ ]:
# Train an SVM model on the BERT embeddings
svm_model = SVC(kernel='linear')
svm_model.fit(train_embeddings, train_labels)

In [ ]:
# Make predictions on the test set
test_predictions = svm_model.predict(test_embeddings)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(test_labels, test_predictions)
print(f"Accuracy: {accuracy}")
print(classification_report(test_labels, test_predictions, target_names=['non-grooming', 'grooming']))

In [ ]:
# Function to determine if a conversation is grooming
def is_grooming_conversation(texts, threshold=0.5):
    embeddings = encode_texts(texts)
    predictions = svm_model.predict(embeddings)
    grooming_count = sum(predictions)
    if grooming_count / len(texts) > threshold:
        return True
    return False

In [ ]:
# Example usage
conversation = [
    "Hey, how are you?"
]

In [ ]:
if is_grooming_conversation(conversation):
    print("The conversation is classified as grooming.")
else:
    print("The conversation is classified as non-grooming.")